In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install -U langchain-mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.9/415.9 kB 8.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25


In [48]:
!pip install kneed

In [51]:
import getpass
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from kneed import KneeLocator
from sklearn.metrics import silhouette_score

In [ ]:
if not os.environ.get("MISTRAL_API_KEY"):
  os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral AI: ")

In [20]:
from langchain_mistralai import MistralAIEmbeddings

embed_llm = MistralAIEmbeddings(
    model="mistral-embed",
)

In [34]:
data = pd.read_csv("/kaggle/input/employees-feedback/diw_data.csv")
data = list(data['feedback'])

In [37]:
embeddings = [embed_llm.embed_query(i) for i in data]

In [65]:
inertia = []
silhouette_scores = {}
K = range(2, 20)  
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)  
    labels = kmeans.fit_predict(embeddings)  
    score = silhouette_score(embeddings, labels)  
    silhouette_scores[k] = score
    inertia.append(kmeans.inertia_)

In [66]:
best_k_silhouette = max(silhouette_scores, key=silhouette_scores.get)


k_values = list(range(2, len(inertia) + 2))
knee_locator = KneeLocator(k_values, inertia, curve="convex", direction="decreasing")
elbow_k = knee_locator.elbow

print(f'Best K using Silhouette Score: {best_k_silhouette}')
print(f'Best K using Elbow Method: {elbow_k}')

Best K using Silhouette Score: 19
Best K using Elbow Method: 10


In [67]:
kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)  
labels = kmeans.fit_predict(embeddings)

In [82]:
similar = [data[i] for i in range(len(embeddings)) if labels[i] == 1]

In [83]:
similar

['Workplace politics are affecting productivity.',
 'Workplace politics are affecting productivity.',
 'Workplace politics are affecting productivity.',
 'Workplace politics are affecting productivity.',
 'The work environment feels toxic and unsupportive.',
 'The work environment feels toxic and unsupportive.',
 'The work environment feels toxic and unsupportive.',
 'The work environment feels toxic and unsupportive.',
 'Workplace politics are affecting productivity.']

In [70]:
pd.DataFrame(labels).value_counts()

0
0    35
7    11
8    10
1     9
4     7
9     7
3     6
5     6
2     5
6     4
Name: count, dtype: int64